# Carbon in the ocean: practical

## Import the data

This first cell below imports the data you will use.  It creates an xarray Dataset called `surface`, which contains a 2D grid (latitude vs longitude) of some different properties at the surface of the ocean.  These data come from the GLODAP compilation; you can find more information about this at [glodap.info](https://www.glodap.info/).

You need to run the code cell (with Ctrl + Enter) once at the start and understand roughly what is going on, but nothing here needs to be changed.

In [ ]:
# Import packages
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np, xarray as xr, PyCO2SYS as pyco2, area
from matplotlib import pyplot as plt
from vufuncs import seawater_1atm_MP81

# Import GLODAP gridded & mapped data
glodap = xr.Dataset()
glodap['alkalinity'] = xr.open_dataarray('data/GLODAPv2.2016b.TAlk_only.nc')
glodap['dic'] = xr.open_dataarray('data/GLODAPv2.2016b.TCO2_only.nc')
glodap['nitrate'] = xr.open_dataarray('data/GLODAPv2.2016b.NO3_only.nc')
glodap['salinity'] = xr.open_dataarray('data/GLODAPv2.2016b.salinity_only.nc')
glodap['temperature'] = xr.open_dataarray('data/GLODAPv2.2016b.temperature_only.nc')

# Pick out surface data
surface = glodap.bfill(dim='depth_surface').isel(depth_surface=0)

# Import climatological mean mixed layer depth and interpolate it to match the GLODAP grid
surface['mld'] = xr.open_dataarray('data/mld_mean.nc').interp_like(surface)

# Calculate surface seawater density in g/cm³
surface['density'] = seawater_1atm_MP81(temperature=surface.temperature, salinity=surface.salinity)

# Calculate surface area of each grid cell in m²
surface_area = np.full_like(surface.lat, np.nan)
for i, lat_i in enumerate(surface.lat):
    surface_area[i] = area.area({
        'type': 'Polygon',
        'coordinates': [[[0, lat_i + 0.5], [1, lat_i + 0.5], [1, lat_i - 0.5], [0, lat_i - 0.5]]],
    })
surface['area'] = (('lat', 'lon'), (surface_area * np.ones_like(surface.mld.values).T).T)

# Set surface area to 0 for the land
surface['area'] = surface.area.where(~surface.temperature.isnull(), 0)

# Summarise the surface Dataset below
surface

If the import has worked, you should now see a summary of the `surface` dataset above.  It contains data gridded in latitude (coordinate `lat`) and longitude (coordinate `lon`) for the following variables:

  * `alkalinity` - total alkalinity, in µmol/kg
  * `dic` - dissolved inorganic carbon (DIC), in µmol/kg
  * `nitrate` - dissolved inorganic nitrate (NO₃⁻), in µmol/kg
  * `salinity` - practical salinity
  * `temperature` - seawater temperature, in °C
  * `mld` - mixed layer depth, in m
  * `density` - seawater density, in kg/l
  * `area` - area of the grid cell, with land areas set to zero, in m²
  
## Carry out calculations
  
You can access the data for any one of these variables (e.g. `alkalinity`) using either `surface.alkalinity` or `surface['alkalinity']`.

You can do calculations with the variables using the normal mathematical symbols, including across the two Datasets.  You can keep these as separate variables or insert them into a Dataset using e.g. `surface['new_variable_name'] = ...` (but NOT `surface.new_variable_name = ...`!).  Some examples:

In [ ]:
# Calculate surface nitrate plus alkalinity divided by salinity, as a separate variable
palk_sal = (surface.nitrate + surface.alkalinity) / surface.salinity

# Calculate mixed layer depth multiplied by grid cell area, as a new variable in the surface Dataset
surface['mlv'] = surface.mld * surface.area

xarray also has built in methods for calculating some overall properties of variables.  For example:

In [ ]:
surface.temperature.mean()  # gives the mean temperature
surface.area.sum()  # gives the sum of all the areas

## Visualise the data

### Making maps

You can make a basic map of any variable as follows:

In [ ]:
plt.figure(figsize=(13, 7))  # this line isn't strictly necessary; you can adjust the figsize values
surface.temperature.plot()

You can also do this with new variables you have created - just swap `surface.temperature` with the variable name, e.g.:

In [ ]:
plt.figure(figsize=(13, 7))
palk_sal.plot(vmin=65, vmax=75)  # vmin and vmax control the colorbar range

### Scatter plots

To make a scatter plot between any variables within the `surface` or `deep` dataset (including new ones you have added):

In [ ]:
plt.figure(figsize=(13, 7))
surface.plot.scatter('salinity', 'alkalinity', s=5, hue='dic')

Note that we used the kwarg `hue` above to also colour the scattered points by one of the variables (in this case, DIC).

## Solve the marine carbonate system with PyCO2SYS

We can use the PyCO2SYS package to solve the complete marine carbonate system from any pair of its variables (see [PyCO2SYS.rtfd.io](https://pyco2sys.readthedocs.io/en/latest/) for more info).  We use `pyco2.sys()` to solve the system, which returns a dict containing all the different calculated properties.

The units for DIC and alkalinity in PyCO2SYS are substance content in µmol/kg.  The unit for pCO₂ is µatm (parts per million).

The main variables we will be interested in are seawater pCO₂ (`pCO2`), pH (`pH`) and the saturation states with respect to calcite (`saturation_calcite`) and aragonite (`saturation_aragonite`):

In [ ]:
# Calculate initial surface ocean pCO2 with PyCO2SYS
results = pyco2.sys(
    par1=surface.alkalinity.values,
    par2=surface.dic.values,
    par1_type=1,  # 1 means alkalinity
    par2_type=2,  # 2 means DIC
    temperature=surface.temperature.values,
    salinity=surface.salinity.values,
)

# Recipe to add a single new variable from PyCO2SYS into the surface Dataset
surface['pCO2'] = (('lat', 'lon'), results['pCO2'])

# Loop using the recipe above to transfer several variables across at once:
for var in ['pH', 'saturation_calcite', 'saturation_aragonite']:
    surface[var] = (('lat', 'lon'), results[var])
    
# Remind ourselves of the contents of surface
surface

You can also use other pairs of known CO₂ system variables in PyCO2SYS.  For example, if we knew alkalinity and pCO₂ and want to find out the corresponding DIC, we could do:

In [ ]:
# Run PyCO2SYS with known alkalinity and pCO₂
results_2 = pyco2.sys(
    par1=surface.alkalinity.values,
    par2=surface.pCO2.values,
    par1_type=1,  # 1 means alkalinity
    par2_type=4,  # 4 means pCO₂
    temperature=surface.temperature.values,
    salinity=surface.salinity.values,
)

# Extract DIC
dic = results_2['dic']

## Geoengineering options

Several ways to encourage the ocean to take up more CO₂ from the ocean have been proposed.  We can use the `surface` dataset imported above to investigate the potential of these to absorb CO₂ and their limitations.

**Iron fertilisation** has been suggested for high-nutrient, low-chlorophyll regions like the Southern Ocean.  Here, major nutrients (e.g. nitrate) are abundant because biological productivity is instead limited by the trace nutrient iron.  Adding iron in these regions might allow the nitrate to be all used up by new primary production, converting DIC into organic matter in the process.

**Ocean alkalinisation** or **liming** involves dissolving minerals like CaCO₃ or olivine into seawater.  These minerals increase alkalinity and thus enhance seawater's CO₂ storage capacity.  This method could be applied anywhere over the surface ocean.

### Questions

If a question requires a numerical answer, use `print()` to show it.

If a question asks for a figure, write the code for the figure.

If a question asks for some text, write it in the code box or in a separate markdown cell.

There are marks for your working, not only the final answers, so show everything!  If you know what calculation you need to do for a question, but don't know how to do this in Python, you will still get some credit if you explain correctly what the calculation should be.  If you have an incorrect answer and then use this for a calculation in a later question, you will not be penalised twice (if the later calculation is otherwise correct).

#### Ocean fertilisation

1. In total, how much (a) nitrate and (b) DIC is there in the surface mixed layer of the global ocean (in moles)?  Assume that the values in `surface` (in μmol/kg) are uniform down to the mixed layer depth.

In [ ]:
# Write code for Q1 here


  2. If all of this nitrate was converted into organic matter (OM), how much DIC would be taken up?  You may assume that OM contains carbon (all from DIC) and nitrogen (all from nitrate) with a C:N ratio of 117:16, following [Anderson & Sarmiento (1994)](https://agupubs.onlinelibrary.wiley.com/doi/10.1029/93GB03318).

In [ ]:
# Write code for Q2 here


  3. If the ocean then replaced all the DIC that was converted into OM in Q2 by absorbing an equal amount of CO₂ from the atmosphere, what is this as a multiple of annual anthropogenic CO₂ emissions (find an appropriate value from [Friedlingstein et al. (2020)](https://essd.copernicus.org/articles/12/3269/2020/))?  In reality, would we expect more or less CO₂ than this actually to be taken up, and why? 

In [ ]:
# Write code/answer for Q3 here


#### Ocean liming

4. Let us dissolve 333 g of CaCO₃ into each m² of the surface ocean and assume this mixes uniformly through the surface mixed layer.  How many moles of (a) DIC and (b) alkalinity have been added to the surface mixed layer immediately after dissolution?

In [ ]:
# Write code for Q4 here


5. What will be the new pCO₂ of the surface layer immediately after dissolution?  Show your answer by drawing a map of the change in pCO₂.

In [ ]:
# Write code for Q5 here


6. If the surface ocean now equilibrates with the atmosphere to return to its original pCO₂ from before we dissolved any extra CaCO₃, how much DIC in total is there now in the surface mixed layer, in moles?

In [ ]:
# Write code for Q6 here


7. How much CO₂ has been transferred between the atmosphere and ocean due to dissolution of CaCO₃ and subsequent gas exchange?  What is the direction of transfer — did the atmosphere gain or lose CO₂?  How many years of annual anthropogenic emissions does this represent?

In [ ]:
# Write code for Q7 here


8. How much CaCO₃ did we dissolve in total?  Put the amount into a unit or context that you might see on a news report, that a layperson could easily grasp.

In [ ]:
# Write code/answer for Q8 here


9. Explain the key theoretical problem with the plan to dissolve CaCO₃ in the surface ocean and draw a map to illustrate it.  Even if we could overcome this and elevate alkalinity by the calculated amount, then are there any reasons why the ocean might not take up the calculated amount of CO₂ from the atmosphere?

In [ ]:
# Write code/answer for Q9 here
